In [27]:
!pip install mysql-connector-python neo4j
!pip install tqdm
!pip install pandas pymysql

/bin/bash: /opt/anaconda/envs/Python3/lib/libtinfo.so.6: no version information available (required by /bin/bash)

         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

bash: /opt/anaconda/envs/Python3/lib/libtinfo.so.6: no version information available (required by bash)
/bin/bash: /opt/anaconda/envs/Python3/lib/libtinfo.so.6: no version information available (required by /bin/bash

In [40]:
import mysql.connector
from neo4j import GraphDatabase
import tqdm
from tqdm import tqdm
import pymysql

# MySQL connection
mysql_connection = mysql.connector.connect(
    host="",
    user="",
    password="",
    database=""
)

# Neo4j connection
neo4j_connection = GraphDatabase.driver(
    uri="",
    auth=("")
)

In [34]:
# Fetch data from MySQL
with mysql_connection.cursor() as cursor:
    cursor.execute("SELECT id, title, instructions FROM recipe")
    recipes = cursor.fetchall()
    cursor.execute("SELECT id, name FROM ingredient")
    ingredients = cursor.fetchall()
    cursor.execute("SELECT recipe_id, ingredient_id, quantity, unit FROM recipe_ingredient")
    recipe_ingredients = cursor.fetchall()

In [25]:
matching_ingredients_query = """
    SELECT i1.name, i2.name
    FROM ingredient i1
    JOIN ingredient i2 ON i1.name = i2.name AND i1.id <> i2.id
    GROUP BY i1.name, i2.name
"""

with mysql_connection.cursor() as cursor:
    cursor.execute(matching_ingredients_query)
    matching_ingredients = cursor.fetchall()

if len(matching_ingredients) > 0:
    print("Matching ingredients found:")
    for ingredient_pair in matching_ingredients:
        print(f"{ingredient_pair[0]} - {ingredient_pair[1]}")
else:
    print("No matching ingredients found.")


Matching ingredients found:
4 tablespoons ceylon loose leaf tea (see recipe note) - 4 tablespoons ceylon loose leaf tea (see recipe note)
1 (3-inch) cinnamon stick - 1 (3-inch) cinnamon stick
1/8 teaspoon ground cardamom - 1/8 teaspoon ground cardamom
1 whole star anise pod - 1 whole star anise pod
1 teaspoon vanilla paste, or 1 vanilla bean, halved lengthwise and scraped - 1 teaspoon vanilla paste, or 1 vanilla bean, halved lengthwise and scraped
1/2 teaspoon ground turmeric - 1/2 teaspoon ground turmeric
1 tablespoon sugar - 1 tablespoon sugar
1/2 cup sweetened condensed milk - 1/2 cup sweetened condensed milk
3 tablespoons whole milk - 3 tablespoons whole milk
1 (8-ounce) box dried manicotti shells (about 12) - 1 (8-ounce) box dried manicotti shells (about 12)
1 3/4 pounds whole or skim milk ricotta (3 1/2 cups) - 1 3/4 pounds whole or skim milk ricotta (3 1/2 cups)
2 large egg yolks - 2 large egg yolks
8 ounces (2 cups) shredded mozzarella, divided - 8 ounces (2 cups) shredded mozz

In [36]:
import pandas as pd

# Load the ingredients table into a pandas DataFrame
query = "SELECT * FROM ingredient"
ingredients_df = pd.read_sql(query, mysql_connection)

# Display the first 10 rows of the DataFrame
print(ingredients_df.head(10))

# Convert the DataFrame to a CSV file
ingredients_df.to_csv('ingredients.csv', index=False)

   id                                               name
0   1  4 tablespoons ceylon loose leaf tea (see recip...
1   2                          1 (3-inch) cinnamon stick
2   3                       1/8 teaspoon ground cardamom
3   4                             1 whole star anise pod
4   5  1 teaspoon vanilla paste, or 1 vanilla bean, h...
5   6                       1/2 teaspoon ground turmeric
6   7                                 1 tablespoon sugar
7   8                   1/2 cup sweetened condensed milk
8   9                           3 tablespoons whole milk
9  10  1 (8-ounce) box dried manicotti shells (about 12)


/tmp/ipykernel_123630/2115724555.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ingredients_df = pd.read_sql(query, mysql_connection)


In [41]:
import re
import pymysql

# The improved function
def extract_raw_ingredient(line):
    pattern = r'([\d\.,\s/]*\w*\s*\w+[\d\.,\s/]*)(\s*\S.*)'
    match = re.match(pattern, line)
    if match:
        return match.group(1).strip(), match.group(2).strip()
    else:
        return None, None
    
# Get the ingredients from the MySQL table
with mysql_connection.cursor() as cursor:
    cursor.execute("SELECT * FROM ingredient")
    ingredients = cursor.fetchall()

# Process the ingredients and update the MySQL table
with mysql_connection.cursor() as cursor:
    for ingredient in ingredients:
        ingredient_id, raw_ingredient = ingredient
        quantity, description = extract_raw_ingredient(raw_ingredient)
        if quantity and description:
            sql = "UPDATE ingredient SET quantity = %s, description = %s WHERE id = %s"
            cursor.execute(sql, (quantity, description, ingredient_id))


ProgrammingError: 1054 (42S22): Unknown column 'quantity' in 'field list'

In [38]:
# Insert data into Neo4j
with neo4j_connection.session() as session:
    # Create Recipe nodes
    for recipe in tqdm(recipes, desc="Inserting Recipe nodes"):
        session.run("""
            MERGE (r:Recipe {id: $id, title: $title, instructions: coalesce($instructions, '')})
        """, {"id": recipe[0], "title": recipe[1], "instructions": recipe[2]})

    # Create Ingredient nodes
    for ingredient in tqdm(ingredients, desc="Inserting Ingredient nodes"):
        session.run("""
            MERGE (i:Ingredient {name: $name})
            ON CREATE SET i.id = $id
        """, {"id": ingredient[0], "name": ingredient[1]})

    # Create relationships between Recipes and Ingredients
    for recipe_ingredient in tqdm(recipe_ingredients, desc="Creating relationships"):
        session.run("""
            MATCH (r:Recipe {id: $recipe_id}), (i:Ingredient {name: $ingredient_name})
            MERGE (r)-[:CONTAINS {quantity: coalesce($quantity, 0), unit: coalesce($unit, '')}]->(i)
        """, {"recipe_id": recipe_ingredient[0], "ingredient_name": recipe_ingredient[1],
              "quantity": recipe_ingredient[2], "unit": recipe_ingredient[3]})

Creating relationships:  31%|███       | 898/2932 [00:20<00:46, 44.20it/s]


KeyboardInterrupt: 

In [15]:
# Close the connections
mysql_connection.close()
neo4j_connection.close()